In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure
import pandas as pd
from pprint import pprint

In [2]:
import holidays

In [3]:
from datetime import datetime,date,time,timedelta

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


#### Load dataset path here

In [5]:
path_to_ds = '../dataset'

In [6]:
# df = pd.read_csv('../dataset/stitched_df.csv').dropna()
# df['ds'] = pd.to_datetime(df['ds'])

In [7]:
# df = df[['ds','y','w1','w2']]

In [8]:
# start_date = [
#     datetime(2018,6,7,11,0,0),datetime(2018,7,3,8,55),datetime(2018,7,3,10,41),datetime(2019,2,12,13,35),datetime(2019,2,17,20,33),
#     datetime(2019,4,3,10,29),datetime(2019,4,11,6,54),datetime(2019,4,17,19,0),datetime(2019,4,17,20,13),datetime(2018,8,13,17,20),
#     datetime(2018,12,30,15,0),datetime(2018,11,15,8,26),
# ]
# end_date = [
#     datetime(2018,6,7,13,30),datetime(2018,7,3,10,39),datetime(2018,7,3,12,57),datetime(2019,2,12,14,41),datetime(2019,2,17,21,50),
#     datetime(2019,4,3,12,10),datetime(2019,4,11,7,35),datetime(2019,4,17,20,11,0),datetime(2019,4,17,21,0),datetime(2018,8,13,19,37),
#     datetime(2018,12,30,15,40),datetime(2018,11,15,8,39),
# ]

In [9]:
# mask = df['ds'] < datetime(2017,2,2,2,2)
# for i in range(len(start_date)):
#     mask = mask | ((df['ds'] >= start_date[i]) & ( df['ds'] <=  end_date[i] ))

In [10]:
# ndf = df.copy()
# ndf.loc[mask,'y'] =  np.nan
# ndf.interpolate(inplace = True)
# ndf = ndf[mask]

In [11]:
# ndf['yhat'] = ndf['y'] + 2*np.random.normal(0,1,len(ndf['y']))

In [12]:
# # output_notebook()
# TOOLS = 'save,box_zoom,reset,wheel_zoom,pan'
# p = figure(title="OPM", y_axis_type="linear", plot_height = 800,x_axis_type="datetime",
#            tools = TOOLS, plot_width = 900)
# p.xaxis.axis_label = 'Time'
# p.yaxis.axis_label = 'OPM'
# p.line(df['ds'],df['y'],color = 'green',line_cap = 'round')
# # p.circle(ndf['ds'],ndf['yhat'],color = 'orange',line_cap = 'round' )
# # p.line(ndf['ds'],ndf['yhat'],color = 'orange',line_cap = 'round' )
# # p.circle(ndf['ds'],ndf['y'],color = 'red', line_cap = 'round')
# # p.line(ndf['ds'],ndf['y'],color = 'red', line_cap = 'round')
# show(p)

In [13]:
# df.loc[mask,'y'] = ndf['yhat']

In [14]:
# df.to_csv('../dataset/clean_df.csv',index=False)

In [15]:
ndf = pd.read_csv(path_to_ds+'/clean_df.csv').dropna()
ndf['ds'] = pd.to_datetime(ndf['ds'])

In [16]:
cdf = ndf.copy()

In [17]:
cdf = cdf[ ['ds','y','w1','w2'] ]

In [18]:
cdf.columns

Index(['ds', 'y', 'w1', 'w2'], dtype='object')

## Encoder Decoder Feature Extraction

##### ```uk_holidays```: holidays of UK

In [19]:
uk_holidays = holidays.CountryHoliday('UK')

In [20]:
cdf['inh'] = cdf['ds'].dt.date.apply( lambda x : 1 if x in uk_holidays else 0)

In [21]:
# uk_holidays

In [22]:
cdf['dt'] =  cdf['ds']
cdf.set_index('ds',inplace = True)

In [23]:
cdf.head()

,y,w1,w2,inh,dt
ds,,,,,
2018-05-23 10:37:00,60.6,52.2,51.8,0,2018-05-23 10:37:00
2018-05-23 10:38:00,60.6,52.6,50.0,0,2018-05-23 10:38:00
2018-05-23 10:39:00,64.4,51.8,57.2,0,2018-05-23 10:39:00
2018-05-23 10:40:00,60.4,54.6,57.2,0,2018-05-23 10:40:00
2018-05-23 10:41:00,58.8,52.2,55.8,0,2018-05-23 10:41:00


#### ```look_back```: number days to take data from the past

In [24]:
look_back = 28

In [25]:
val_cols =  []
nh_cols  =  []
for i in reversed(range(1,look_back+1)):
    col = '-%dy'%i
    cdf [col] = cdf['y'].shift(periods = 1,freq = timedelta(days = i ))
    val_cols.append( col )
    col = '-%dinh'%i
    cdf [col] = cdf['inh'].shift(periods = 1,freq = timedelta(days = i ))
    nh_cols.append( col )

#### ```use_next```: number of days to take data from the future

In [26]:
use_next = 7

In [27]:
for i in (range(use_next)):
    sym = '+'
    if i == 0:
        sym = ''
    col = '%s%dy'%(sym,i)
    cdf [col] = cdf['y'].shift(periods = -1,freq = timedelta(days = i ))
    val_cols.append( col )
    col = '%s%dinh'%(sym,i)
    cdf [col] = cdf['inh'].shift(periods = -1,freq = timedelta(days = i ))
    nh_cols.append( col )

In [28]:
feat_cols = list(zip(val_cols,nh_cols))

In [29]:
feat_encoder = []
i = 0
for j in range(-look_back,use_next):
    sym  = ''
    if j > 0:
        sym = '+'
    cols = '%s%dD'%(sym,j)
    cdf[ cols ] =  list( zip(cdf[feat_cols[i][0]],cdf[feat_cols[i][1]]) )
    i = i+ 1
    feat_encoder.append(cols)
#     break

In [30]:
# pprint(cdf.dropna().head(1))
cols_to_preserve = ['dt','y','w1','w2','-28y']+feat_encoder

In [31]:
cdf = cdf[cols_to_preserve]

In [32]:
# cdf.dropna(inplace = True )

In [33]:
X_encoder = cdf.dropna()[feat_encoder[:-7]].values

In [34]:
XX_encoder = [ ]
for i in range(len(X_encoder)):
    XX_encoder.append(np.asarray(list(X_encoder[i])))
XX_encoder = np.array( XX_encoder )
print( 'Encoder Feature ', XX_encoder.shape)

Encoder Feature  (390195, 28, 2)


In [35]:
XX_decoder = [ ]
for i in range(len(X_encoder)):
    XX_decoder.append(np.asarray(list(X_encoder[i][-7:])))
XX_decoder = np.array( XX_decoder )
print( 'Decoder Feature ', XX_decoder.shape)

Decoder Feature  (390195, 7, 2)


In [36]:
Y_decoder = cdf.dropna()[feat_encoder[-7:]].values

In [37]:
YY_decoder = [ ]
for i in range(len(Y_decoder)):
    YY_decoder.append(np.asarray(list(Y_decoder[i])))
YY_decoder = np.array( YY_decoder )
print( 'Decoder Targets ', YY_decoder.shape)

Decoder Targets  (390195, 7, 2)


In [38]:
VAE_dt = list(cdf['dt'].dropna())

In [39]:
X_pre_train = { 'encoderIn':XX_encoder,'decoderIn':XX_decoder}
Y_pre_train = YY_decoder
VAE_data = {'X':X_pre_train, 'Y':Y_pre_train,'dt':VAE_dt}


### Feature Extraction for encoder in inference step

##### ```forecast_next```: Number of days to forecast in the prediction/inference step
should be less than ```use_next```

In [40]:
forecast_next = 1

In [41]:
cols_to_preserve = cols_to_preserve[:-use_next+forecast_next]
cdf = cdf[ cols_to_preserve ].dropna()

In [42]:
X_encoder = cdf.dropna()[feat_encoder[:-7]].values
XX_encoder_inf = [ ]
for i in range(len(X_encoder)):
    XX_encoder_inf.append(np.asarray(list(X_encoder[i])))
XX_encoder_inf = np.array( XX_encoder_inf )
print( 'Encoder Inference Feature ', XX_encoder_inf.shape)

Encoder Inference Feature  (390195, 28, 2)


In [43]:
Y_nn = (cdf.dropna()['y'].values).reshape(-1,1,1)

In [44]:
colors = ['#003f5c','#374c80','#7a5195','#bc5090','#ef5675','#ff764a','#ffa600']
TOOLS = 'save,box_zoom,reset,wheel_zoom,pan'
p = figure(title="OPM", y_axis_type="linear", plot_height = 800,x_axis_type="datetime",
           tools = TOOLS, plot_width = 900)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'OPM'
# p.line(cdf['dt'],cdf['-1y'],color = 'green')
p.line(cdf['dt'],cdf['y'],color = 'blue')
show(p)

In [45]:
cdf.head()

,dt,y,w1,w2,-28y,-28D,-27D,-26D,-25D,-24D,...,-9D,-8D,-7D,-6D,-5D,-4D,-3D,-2D,-1D,0D
ds,,,,,,,,,,,,,,,,,,,,,
2018-06-20 10:37:00,2018-06-20 10:37:00,54.0,50.4,52.4,60.6,"(60.6, 0.0)","(57.6, 0.0)","(60.6, 0.0)","(46.8, 0.0)","(45.6, 0.0)",...,"(60.0, 0.0)","(52.2, 0.0)","(50.4, 0.0)","(53.4, 0.0)","(56.0, 0.0)","(43.2, 0.0)","(40.0, 0.0)","(55.6, 0.0)","(42.6, 0.0)","(54.0, 0)"
2018-06-20 10:38:00,2018-06-20 10:38:00,56.4,50.2,51.2,60.6,"(60.6, 0.0)","(57.2, 0.0)","(60.2, 0.0)","(46.2, 0.0)","(44.0, 0.0)",...,"(60.0, 0.0)","(50.8, 0.0)","(50.2, 0.0)","(54.4, 0.0)","(53.2, 0.0)","(46.2, 0.0)","(40.6, 0.0)","(54.4, 0.0)","(42.6, 0.0)","(56.4, 0)"
2018-06-20 10:39:00,2018-06-20 10:39:00,55.2,49.8,50.0,64.4,"(64.4, 0.0)","(56.6, 0.0)","(58.2, 0.0)","(44.2, 0.0)","(41.8, 0.0)",...,"(56.8, 0.0)","(48.6, 0.0)","(49.8, 0.0)","(55.4, 0.0)","(50.8, 0.0)","(46.0, 0.0)","(40.0, 0.0)","(54.4, 0.0)","(44.8, 0.0)","(55.2, 0)"
2018-06-20 10:40:00,2018-06-20 10:40:00,55.4,48.2,47.6,60.4,"(60.4, 0.0)","(54.4, 0.0)","(56.2, 0.0)","(43.0, 0.0)","(39.2, 0.0)",...,"(57.0, 0.0)","(48.6, 0.0)","(48.2, 0.0)","(57.8, 0.0)","(52.6, 0.0)","(46.4, 0.0)","(41.4, 0.0)","(58.4, 0.0)","(46.6, 0.0)","(55.4, 0)"
2018-06-20 10:41:00,2018-06-20 10:41:00,54.6,51.8,51.4,58.8,"(58.8, 0.0)","(54.2, 0.0)","(50.8, 0.0)","(41.8, 0.0)","(35.6, 0.0)",...,"(55.4, 0.0)","(51.8, 0.0)","(51.8, 0.0)","(59.2, 0.0)","(48.6, 0.0)","(46.8, 0.0)","(40.6, 0.0)","(54.0, 0.0)","(53.0, 0.0)","(54.6, 0)"


## Day of the Week Feat Extraction

In [46]:
d_list = ['day_Monday', 'day_Tuesday', 'day_Wednesday','day_Thursday', 'day_Friday',  'day_Saturday','day_Sunday']
d_list = [ x[4:] for x in d_list]

In [47]:
def symm_diff(curr,other):
    a  = min(curr,other)
    b  = max(curr,other )
    b  = b - a
    if b <= 3  :
        return b
    else:
        return 7 - b
alpha = 0.4
for i in range(len(d_list)) :
    day = d_list[i]
#     print(day,i)
    cdf[day] = cdf['dt'].dt.dayofweek.apply( lambda x : np.exp(-1*((symm_diff(i,x))**2)/(2*alpha)) )

In [48]:
# output_notebook()
colors = ['#003f5c','#374c80','#7a5195','#bc5090','#ef5675','#ff764a','#ffa600']
TOOLS = 'save,box_zoom,reset,wheel_zoom,pan'
p = figure(title="OPM", y_axis_type="linear", plot_height = 800,x_axis_type="datetime",
           tools = TOOLS, plot_width = 900)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'OPM'
for i in range(len(d_list)) :
    day = d_list[i]
    p.line(cdf['dt'],cdf[day],color = colors[i],line_cap = 'round',legend = day)
#     break
# p.circle(ndf['ds'],ndf['yhat'],color = 'orange',line_cap = 'round' )
# p.line(ndf['ds'],ndf['yhat'],color = 'orange',line_cap = 'round' )
# p.circle(ndf['ds'],ndf['y'],color = 'red', line_cap = 'round')
# p.line(ndf['ds'],ndf['y'],color = 'red', line_cap = 'round')
show(p)

In [49]:
cdf['h'] = cdf['dt'].dt.date.apply( lambda x : .8 if x in uk_holidays else  .2 if timedelta(days = 1) + x in uk_holidays or  x - timedelta(days = 1)  in uk_holidays  else  -10000 )

In [50]:
beta = 0.4
cdf['h_rbf'] =  cdf['h'].apply( lambda x : np.exp(-1*((x - .8)**2)/(2*beta))  )

In [51]:
# output_notebook()
colors = ['#003f5c','#374c80','#7a5195','#bc5090','#ef5675','#ff764a','#ffa600']
TOOLS = 'save,box_zoom,reset,wheel_zoom,pan'
p = figure(title="OPM", y_axis_type="linear", plot_height = 800,x_axis_type="datetime",
           tools = TOOLS, plot_width = 900)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'holiday'
p.line( cdf['dt'],cdf['h_rbf'],line_width = 4)
# p.line(ndf['ds'],ndf['y'],color = 'red', line_cap = 'round')
show(p)

In [52]:
exogenous  =  cdf[['h_rbf']+d_list]

In [53]:
exogenous.describe()

,h_rbf,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
count,390195.000000,390195.000000,390195.000000,390195.000000,390195.000000,390195.000000,390195.000000,390195.000000
mean,0.077372,0.224980,0.223759,0.226371,0.227791,0.228249,0.228177,0.227183
std,0.239487,0.338329,0.336933,0.339401,0.339585,0.339776,0.339797,0.339499
min,0.000000,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013
25%,0.000000,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013
50%,0.000000,0.006738,0.006738,0.006738,0.006738,0.006738,0.006738,0.006738
75%,0.000000,0.286505,0.286505,0.286505,0.286505,0.286505,0.286505,0.286505
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [54]:
# exogenous = exoge

In [55]:
X_inference = { 'encoderIn':XX_encoder_inf, 'nn' : exogenous.values }
Y_inf = Y_nn
MLP_data = {'X':X_inference,'Y':Y_inf,'dt':list(cdf['dt'])}

In [56]:
op_data = {'vae':VAE_data,'mlp':MLP_data}

In [58]:
import pickle

In [59]:
file_name = path_to_ds+'/op_data.csv'

In [ ]:
with open(file_name,'wb') as f:
    pickle.dump()